In [1]:
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
import math
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from matplotlib.patches import Rectangle
import os

from mpl_toolkits import mplot3d
import pickle

In [2]:
# path="./COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/Sequence 1/"
class Task3:
    def __init__(self, path):
        self.root=Tk()
        self.frame = Frame(self.root, bd=2, relief=SUNKEN)
        self.frame.grid_rowconfigure(0, weight=1)
        self.frame.grid_columnconfigure(0, weight=1)
        self.label = Label(self.root, text="Click On The Screen")
        self.img_paths=[elem for elem in sorted(os.listdir(path)) if elem.endswith("tif")]
        self.count=0
        self.path=path
        xscroll = Scrollbar(self.frame, orient=HORIZONTAL)
        xscroll.grid(row=1, column=0, sticky=E+W)
        yscroll = Scrollbar(self.frame)
        yscroll.grid(row=0, column=1, sticky=N+S)
        self.canvas = Canvas(self.frame, bd=0, xscrollcommand=xscroll.set, yscrollcommand=yscroll.set)
        self.image_on_canvas=None
        self.trajectory={}
        self.mitosis={}
        
    def load_files(self, path):
        with open(path + 'mitosis.pkl', 'rb') as f:
            self.mitosis=pickle.load(f)
        with open(path + 'trajectory.pkl', 'rb') as f:
            self.trajectory=pickle.load(f)
        
    def detect_cell(self, x, y):
        maxi=1000
        label=None
        for key in self.trajectory.keys():
            
            x1, x2, y1, y2=self.trajectory[key][self.count][1], self.trajectory[key][self.count][2], self.trajectory[key][self.count][3], self.trajectory[key][self.count][4]
            mid_x=(x2-x1)/2+x1
            mid_y=(y2-y1)/2+y1
            if(key==0):
                print(mid_x, mid_y, x, y)
            distance=math.sqrt((mid_x-x)**2+(mid_y-y)**2)
            if distance < maxi:
                maxi=distance
                label=key
            
        return label
        
    
    def speed_cell(self, label):
        if self.count==0:
            return 0
        x1, x2, y1, y2=self.trajectory[label][self.count][1], self.trajectory[label][self.count][2], self.trajectory[label][self.count][3], self.trajectory[label][self.count][4]
        curr_x=(x2-x1)/2+x1
        curr_y=(y2-y1)/2+y1
        x1, x2, y1, y2=self.trajectory[label][self.count-1][1], self.trajectory[label][self.count-1][2], self.trajectory[label][self.count-1][3], self.trajectory[label][self.count-1][4]
        prev_x=(x2-x1)/2+x1
        prev_y=(y2-y1)/2+y1
        return math.sqrt((curr_x-prev_x)**2+(curr_y-prev_y)**2)
    
    def total_distance(self, label):
        if self.count==0:
            return 0
        total=0
        for i in range(1, self.count+1):
            x1, x2, y1, y2=self.trajectory[label][i][1], self.trajectory[label][i][2], self.trajectory[label][i][3], self.trajectory[label][i][4]
            curr_x=(x2-x1)/2+x1
            curr_y=(y2-y1)/2+y1
            x1, x2, y1, y2=self.trajectory[label][i-1][1], self.trajectory[label][i-1][2], self.trajectory[label][i-1][3], self.trajectory[label][i-1][4]
            prev_x=(x2-x1)/2+x1
            prev_y=(y2-y1)/2+y1
            if prev_x==-1 and prev_y==-1:
                continue
            total+=math.sqrt((curr_x-prev_x)**2+(curr_y-prev_y)**2)
            
        return total
    
    def net_distance(self, label):
        for i in range(self.count):
            x1, x2, y1, y2=self.trajectory[label][i][1], self.trajectory[label][i][2], self.trajectory[label][i][3], self.trajectory[label][i][4]
            start_x=(x2-x1)/2+x1
            start_y=(y2-y1)/2+y1
            if start_x!=-1:
                break
        x1, x2, y1, y2=self.trajectory[label][self.count][1], self.trajectory[label][self.count][2], self.trajectory[label][self.count][3], self.trajectory[label][self.count][4]
        curr_x=(x2-x1)/2+x1
        curr_y=(y2-y1)/2+y1
        return math.sqrt((curr_x-start_x)**2+(curr_y-start_y)**2)
    
    def confinement_ratio(self, total, net):
        return total/net

    def printcoords(self, event):
        canvas = event.widget
        x = canvas.canvasx(event.x)
        y = canvas.canvasy(event.y)
        label=self.detect_cell(x, y)
        speed=self.speed_cell(label)
        total=self.total_distance(label)
        net=self.net_distance(label)
        confine=self.confinement_ratio(total, net)
        self.label['text'] = "label: "+str(label)+"\n speed: "+str(speed)+"\n total: "+str(total)+"\n net: "+str(net)+"\n confine: "+str(confine)
        
    def check(self):
        img = ImageTk.PhotoImage(Image.open(self.path+self.img_paths[0]).resize((500, 500)))
        Image.open(self.path+self.img_paths[0])
        self.canvas.grid(row=0, column=0, sticky=N+S+E+W)
        self.image_on_canvas=self.canvas.create_image(0,0,image=img,anchor="nw")
        self.canvas.config(scrollregion=self.canvas.bbox(ALL))
        self.canvas.bind("<Button 1>",self.printcoords)
        self.frame.pack(fill=BOTH,expand=1)
        self.root.after(1000, self.changeimage)
        self.label.pack()
        self.root.mainloop()
          
    def changeimage(self):
        self.count+=1
        if self.count > len(self.img_paths)-1:
            self.root.quit()
            return
        self.image = ImageTk.PhotoImage(Image.open(self.path+self.img_paths[self.count]).resize((500, 500)))
        self.canvas.itemconfig(self.image_on_canvas, image=self.image)
        self.root.after(1000, self.changeimage)

In [3]:
path="mitosis/fluo/Sequence 1/"
task3=Task3(path)
task3.load_files(path)
task3.check()

113.5 4.5 235.0 113.0
112.0 5.5 235.0 114.0
-1.0 -1.0 231.0 155.0


KeyboardInterrupt: 

In [ ]:
[elem for elem in sorted(os.listdir(path)) if elem.endswith("tif")]

In [ ]:
for i in range(1, 1):
    print(i)